In [1]:
%pip install emoji

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
!pip install --user unicodedata2

In [7]:
!pip install wordsegment

Defaulting to user installation because normal site-packages is not writeable


In [8]:
!pip install transformers


Defaulting to user installation because normal site-packages is not writeable


In [9]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable


In [10]:
!pip install torch

Defaulting to user installation because normal site-packages is not writeable


## Import Statements

In [11]:
import json
import csv
import re
import emoji
from wordsegment import load, segment
import unicodedata
import pandas as pd
import nltk
from nltk.corpus import stopwords
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import numpy as np
from scipy import stats
from ast import literal_eval
import torch
from torch.utils.data import Dataset
from torch.utils.data import random_split
from transformers import RobertaModel, RobertaConfig
from transformers import TrainingArguments
from transformers import Trainer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


/N/u/aks14/Carbonate/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Function to preprocess data

In [12]:
def preprocess_data(json_file_path):
    def clean_text(text):
      # Remove usernames
      text = re.sub(r'@[\w]+', '', text)
      # Convert to lowercase
      text = text.lower()
      # Replace emojis with their unicode description
      text = emoji.demojize(text, language='alias')
      # Hashtag segmentation
      text = re.sub(r'#(\w+)', lambda m: ' '.join(segment(m.group(1))), text)
      # Remove links
      text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
      # Remove unwanted '/'
      text = re.sub(r'/\s', '', text)
      # Remove punctuation except ' - / % & ... ¿ ?
      text = re.sub(r'[^\w\s\'%&¿?/-]', '', text)
      # Remove accented characters
      text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
      return text

    # function to remove data rows from dataframe if "yes" and "no" labels for each tweet are tied in label_task1 column
    def filter_tied_rows(row):
      yes_count = row['labels_task1'].count('YES')
      no_count = row['labels_task1'].count('NO')
      
      if yes_count == no_count:
          return False
      return True

    # Load word segment data
    load()

    # Load JSON data
    with open(json_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Extract data and clean tweets
    rows = []
    for key in data:
        item = data[key]
        row = [
            item['id_EXIST'],
            item['lang'],
            clean_text(item['tweet']),
            item['number_annotators'],
            item['gender_annotators'],
            item['age_annotators'],
            item['labels_task1']
        ]
        rows.append(row)
    # Write data to CSV
    with open('data.csv', 'w', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['id_EXIST', 'lang', 'tweet', 'number_annotators', 'gender_annotators', 'age_annotators', 'labels_task1'])
        csvwriter.writerows(rows)

    df_data = pd.read_csv("data.csv")

    # Remove stopwords

    nltk.download('stopwords')

    # Remove stopwords from Spanish tweets
    spanish_stopwords = stopwords.words('spanish')
    df_data.loc[df_data['lang'] == 'es', 'tweet'] = df_data.loc[df_data['lang'] == 'es', 'tweet'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in spanish_stopwords]))

    # Remove stopwords from English tweets
    english_stopwords = stopwords.words('english')
    df_data.loc[df_data['lang'] == 'en', 'tweet'] = df_data.loc[df_data['lang'] == 'en', 'tweet'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in english_stopwords]))

    # storing cleaned dataset as df_cleaned DataFrame
    df_cleaned = df_data[df_data.apply(filter_tied_rows, axis=1)]

    return df_cleaned

###  additional_data variable, which contains lists of strings for gender and age annotators. To resolve this issue, you can encode the gender and age annotators as numerical values. One way to do this is to use one-hot encoding for both gender and age categories.

In [13]:
def process_and_tokenize_data(df_cleaned):
    # Set up tokenizer and model
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

    # Tokenize tweets and create attention masks
    encoded_tweets = tokenizer.batch_encode_plus(
        df_cleaned['tweet'].tolist(),
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    # Define the mapping for string labels
    label_mapping = {'YES': 1, 'NO': 0}
    # Convert string representation of lists into lists of strings
    string_lists = df_cleaned['labels_task1'].apply(literal_eval)
    # Convert lists of strings into lists of integers
    integer_labels = string_lists.apply(lambda x: [label_mapping[label] for label in x])
    # Get the mode (most frequent value) of each list as the label
    labels = integer_labels.apply(lambda x: stats.mode(x)[0][0]).to_numpy()


    # Encode annotations
    gender_dummies = pd.get_dummies(df_cleaned['gender_annotators'].apply(pd.Series).stack()).sum(level=0)
    age_dummies = pd.get_dummies(df_cleaned['age_annotators'].apply(pd.Series).stack()).sum(level=0)
    # Concatenate the one-hot encoded DataFrames
    encoded_annotations = pd.concat([gender_dummies, age_dummies], axis=1)
    # Convert each row of the encoded_annotations DataFrame to a tensor
    encoded_annotations_list = [torch.tensor(row.values, dtype=torch.float) for _, row in encoded_annotations.iterrows()]

    return encoded_tweets, labels, encoded_annotations_list


### CustomDataset class to accept the additional data and include it in the __getitem__ method. It concatenates the additional data with the input embeddings before passing it to the model. 



In [14]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels, additional_data: pd.DataFrame):
        self.encodings = encodings
        self.labels = labels
        self.additional_data = additional_data

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         item['labels'] = torch.tensor(self.labels.iloc[idx], dtype=torch.long)
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        print(f"Index: {idx}")
        item['additional_data'] = torch.tensor(self.additional_data.loc[idx]['encoded_annotations'], dtype=torch.float32)
        print(f"Additional data shape: {self.additional_data.loc[idx]['encoded_annotations'].shape}")
        return item

    def __len__(self):
        return len(self.labels)

## Training Data

In [15]:
json_file_path = r"/N/u/aks14/Carbonate/Downloads/Applying_MLinCL_project/training/EXIST2023_training.json"
df_training_cleaned = preprocess_data(json_file_path)
df_training_cleaned.info()


[nltk_data] Downloading package stopwords to
[nltk_data]     /N/u/aks14/Carbonate/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6064 entries, 0 to 6919
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id_EXIST           6064 non-null   int64 
 1   lang               6064 non-null   object
 2   tweet              6064 non-null   object
 3   number_annotators  6064 non-null   int64 
 4   gender_annotators  6064 non-null   object
 5   age_annotators     6064 non-null   object
 6   labels_task1       6064 non-null   object
dtypes: int64(2), object(5)
memory usage: 379.0+ KB


In [16]:
encoded_tweets, labels, encoded_annotations = process_and_tokenize_data(df_training_cleaned)
# Add the list of tensors to  dataset
df_training_cleaned['encoded_annotations'] = encoded_annotations
df_training_cleaned.head()
df_training_cleaned.reset_index(drop=True, inplace=True)
training_dataset = CustomDataset(encoded_tweets, labels, df_training_cleaned)

/tmp/ipykernel_12357/1385765932.py:26: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  gender_dummies = pd.get_dummies(df_cleaned['gender_annotators'].apply(pd.Series).stack()).sum(level=0)
/tmp/ipykernel_12357/1385765932.py:27: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  age_dummies = pd.get_dummies(df_cleaned['age_annotators'].apply(pd.Series).stack()).sum(level=0)


## Dev Dataset

In [17]:
json_file_path = r"/N/u/aks14/Carbonate/Downloads/Applying_MLinCL_project/dev/EXIST2023_dev.json"
df_dev_cleaned = preprocess_data(json_file_path)
df_dev_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 934 entries, 1 to 1037
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id_EXIST           934 non-null    int64 
 1   lang               934 non-null    object
 2   tweet              934 non-null    object
 3   number_annotators  934 non-null    int64 
 4   gender_annotators  934 non-null    object
 5   age_annotators     934 non-null    object
 6   labels_task1       934 non-null    object
dtypes: int64(2), object(5)
memory usage: 58.4+ KB


[nltk_data] Downloading package stopwords to
[nltk_data]     /N/u/aks14/Carbonate/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
encoded_tweets, labels, encoded_annotations = process_and_tokenize_data(df_dev_cleaned)
# Add the list of tensors to  dataset
df_dev_cleaned['encoded_annotations'] = encoded_annotations
df_dev_cleaned.head()
df_dev_cleaned.reset_index(drop=True, inplace=True)
dev_dataset = CustomDataset(encoded_tweets, labels, df_dev_cleaned)

/tmp/ipykernel_12357/1385765932.py:26: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  gender_dummies = pd.get_dummies(df_cleaned['gender_annotators'].apply(pd.Series).stack()).sum(level=0)
/tmp/ipykernel_12357/1385765932.py:27: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  age_dummies = pd.get_dummies(df_cleaned['age_annotators'].apply(pd.Series).stack()).sum(level=0)


### Modifying the model to accept the additional data. Creating a new custom class that inherits from RobertaForSequenceClassification

In [19]:
class CustomRobertaForSequenceClassification(RobertaModel):
    def __init__(self, config: RobertaConfig):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)
        self.classifier = torch.nn.Linear(config.hidden_size + 4, config.num_labels)
        
        self.init_weights()

    def forward(self, input_ids, additional_data=None, attention_mask=None, labels=None):
        outputs = super().forward(input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs[0]
        cls_output = last_hidden_state[:, 0]
        print(f"CLS output shape: {cls_output.shape}")
        print(f"Additional data shape: {additional_data.shape}")

        if additional_data is not None:
            combined_output = torch.cat((cls_output, additional_data), dim=1)
            print(f"Combined output shape: {combined_output.shape}")
        else:
            combined_output = cls_output

        logits = self.classifier(self.dropout(combined_output))

        loss = None
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return (loss, logits) if labels is not None else (logits,)


# # Instantiate the model
model = CustomRobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

Some weights of the model checkpoint at roberta-base were not used when initializing CustomRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing CustomRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Define the training arguments

In [45]:
training_args = TrainingArguments(
    output_dir="/N/u/aks14/Carbonate/Downloads/Applying_MLinCL_project/results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
)


### using the Hugging Face compute_metrics function to extract accuracy, precision, and other metrics by 

In [46]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }


### Custom trainer class to pass the additional data to the model

In [47]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        additional_data = inputs.pop("additional_data")
        outputs = model(**inputs, additional_data=additional_data, labels=labels)
        loss = outputs[0] if isinstance(outputs, tuple) else outputs
        return (loss, outputs) if return_outputs else loss

# Set up the custom trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=training_dataset,
    eval_dataset=dev_dataset,
    compute_metrics=compute_metrics
)


### Train the model

In [48]:
trainer.train()

/N/u/aks14/Carbonate/.local/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/tmp/ipykernel_12357/1949724228.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_12357/1949724228.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['additional_data'] = torch.tensor(self.additional_data.loc[idx]['encoded_annotations'], dtype=torch.float32)


Index: 2056
Additional data shape: torch.Size([4])
Index: 3110
Additional data shape: torch.Size([4])
Index: 1936
Additional data shape: torch.Size([4])
Index: 5589
Additional data shape: torch.Size([4])
Index: 5588
Additional data shape: torch.Size([4])
Index: 249
Additional data shape: torch.Size([4])
Index: 3130
Additional data shape: torch.Size([4])
Index: 5520
Additional data shape: torch.Size([4])
Index: 3383
Additional data shape: torch.Size([4])
Index: 4062
Additional data shape: torch.Size([4])
Index: 2016
Additional data shape: torch.Size([4])
Index: 3629
Additional data shape: torch.Size([4])
Index: 5865
Additional data shape: torch.Size([4])
Index: 2206
Additional data shape: torch.Size([4])
Index: 5997
Additional data shape: torch.Size([4])
Index: 1545
Additional data shape: torch.Size([4])
CLS output shape: torch.Size([16, 768])
Additional data shape: torch.Size([16, 4])
Combined output shape: torch.Size([16, 772])


Step,Training Loss
500,0.395400
1000,0.378400


Index: 3527
Additional data shape: torch.Size([4])
Index: 4972
Additional data shape: torch.Size([4])
Index: 5378
Additional data shape: torch.Size([4])
Index: 1877
Additional data shape: torch.Size([4])
Index: 1441
Additional data shape: torch.Size([4])
Index: 5714
Additional data shape: torch.Size([4])
Index: 2999
Additional data shape: torch.Size([4])
Index: 4618
Additional data shape: torch.Size([4])
Index: 5
Additional data shape: torch.Size([4])
Index: 933
Additional data shape: torch.Size([4])
Index: 2564
Additional data shape: torch.Size([4])
Index: 1615
Additional data shape: torch.Size([4])
Index: 4046
Additional data shape: torch.Size([4])
Index: 851
Additional data shape: torch.Size([4])
Index: 2043
Additional data shape: torch.Size([4])
Index: 2717
Additional data shape: torch.Size([4])
CLS output shape: torch.Size([16, 768])
Additional data shape: torch.Size([16, 4])
Combined output shape: torch.Size([16, 772])
Index: 4250
Additional data shape: torch.Size([4])
Index: 604

CLS output shape: torch.Size([16, 768])
Additional data shape: torch.Size([16, 4])
Combined output shape: torch.Size([16, 772])
Index: 1094
Additional data shape: torch.Size([4])
Index: 8
Additional data shape: torch.Size([4])
Index: 3714
Additional data shape: torch.Size([4])
Index: 5719
Additional data shape: torch.Size([4])
Index: 5716
Additional data shape: torch.Size([4])
Index: 5109
Additional data shape: torch.Size([4])
Index: 1514
Additional data shape: torch.Size([4])
Index: 3104
Additional data shape: torch.Size([4])
Index: 3676
Additional data shape: torch.Size([4])
Index: 236
Additional data shape: torch.Size([4])
Index: 940
Additional data shape: torch.Size([4])
Index: 3803
Additional data shape: torch.Size([4])
Index: 2002
Additional data shape: torch.Size([4])
Index: 680
Additional data shape: torch.Size([4])
Index: 3489
Additional data shape: torch.Size([4])
Index: 568
Additional data shape: torch.Size([4])
CLS output shape: torch.Size([16, 768])
Additional data shape: 

...

CLS output shape: torch.Size([16, 768])
Additional data shape: torch.Size([16, 4])
Combined output shape: torch.Size([16, 772])
Index: 5320
Additional data shape: torch.Size([4])
Index: 5403
Additional data shape: torch.Size([4])
Index: 5452
Additional data shape: torch.Size([4])
Index: 46
Additional data shape: torch.Size([4])
Index: 5846
Additional data shape: torch.Size([4])
Index: 2318
Additional data shape: torch.Size([4])
Index: 3900
Additional data shape: torch.Size([4])
Index: 3464
Additional data shape: torch.Size([4])
Index: 1991
Additional data shape: torch.Size([4])
Index: 1900
Additional data shape: torch.Size([4])
Index: 1518
Additional data shape: torch.Size([4])
Index: 5802
Additional data shape: torch.Size([4])
Index: 2310
Additional data shape: torch.Size([4])
Index: 732
Additional data shape: torch.Size([4])
Index: 3731
Additional data shape: torch.Size([4])
Index: 1061
Additional data shape: torch.Size([4])
CLS output shape: torch.Size([16, 768])
Additional data sha

CLS output shape: torch.Size([16, 768])
Additional data shape: torch.Size([16, 4])
Combined output shape: torch.Size([16, 772])
Index: 4776
Additional data shape: torch.Size([4])
Index: 354
Additional data shape: torch.Size([4])
Index: 2765
Additional data shape: torch.Size([4])
Index: 2803
Additional data shape: torch.Size([4])
Index: 3571
Additional data shape: torch.Size([4])
Index: 4475
Additional data shape: torch.Size([4])
Index: 4601
Additional data shape: torch.Size([4])
Index: 5611
Additional data shape: torch.Size([4])
Index: 2277
Additional data shape: torch.Size([4])
Index: 5133
Additional data shape: torch.Size([4])
Index: 3866
Additional data shape: torch.Size([4])
Index: 4096
Additional data shape: torch.Size([4])
Index: 1156
Additional data shape: torch.Size([4])
Index: 1964
Additional data shape: torch.Size([4])
Index: 3515
Additional data shape: torch.Size([4])
Index: 1850
Additional data shape: torch.Size([4])
CLS output shape: torch.Size([16, 768])
Additional data s

CLS output shape: torch.Size([16, 768])
Additional data shape: torch.Size([16, 4])
Combined output shape: torch.Size([16, 772])
Index: 273
Additional data shape: torch.Size([4])
Index: 5945
Additional data shape: torch.Size([4])
Index: 1885
Additional data shape: torch.Size([4])
Index: 1675
Additional data shape: torch.Size([4])
Index: 5022
Additional data shape: torch.Size([4])
Index: 2460
Additional data shape: torch.Size([4])
Index: 4631
Additional data shape: torch.Size([4])
Index: 1951
Additional data shape: torch.Size([4])
Index: 3706
Additional data shape: torch.Size([4])
Index: 2918
Additional data shape: torch.Size([4])
Index: 5597
Additional data shape: torch.Size([4])
Index: 3294
Additional data shape: torch.Size([4])
Index: 3051
Additional data shape: torch.Size([4])
Index: 5900
Additional data shape: torch.Size([4])
Index: 4075
Additional data shape: torch.Size([4])
Index: 5075
Additional data shape: torch.Size([4])
CLS output shape: torch.Size([16, 768])
Additional data s

CLS output shape: torch.Size([16, 768])
Additional data shape: torch.Size([16, 4])
Combined output shape: torch.Size([16, 772])
Index: 4403
Additional data shape: torch.Size([4])
Index: 974
Additional data shape: torch.Size([4])
Index: 168
Additional data shape: torch.Size([4])
Index: 5
Additional data shape: torch.Size([4])
Index: 4928
Additional data shape: torch.Size([4])
Index: 1251
Additional data shape: torch.Size([4])
Index: 4655
Additional data shape: torch.Size([4])
Index: 5203
Additional data shape: torch.Size([4])
Index: 1185
Additional data shape: torch.Size([4])
Index: 3798
Additional data shape: torch.Size([4])
Index: 2004
Additional data shape: torch.Size([4])
Index: 2923
Additional data shape: torch.Size([4])
Index: 1510
Additional data shape: torch.Size([4])
Index: 1691
Additional data shape: torch.Size([4])
Index: 5414
Additional data shape: torch.Size([4])
Index: 4124
Additional data shape: torch.Size([4])
CLS output shape: torch.Size([16, 768])
Additional data shape

TrainOutput(global_step=1137, training_loss=0.37541031648741546, metrics={'train_runtime': 9405.6864, 'train_samples_per_second': 1.934, 'train_steps_per_second': 0.121, 'total_flos': 1196629191548928.0, 'train_loss': 0.37541031648741546, 'epoch': 3.0})

In [50]:
model.save_pretrained(r"/N/u/aks14/Carbonate/Downloads/Applying_MLinCL_project")

In [ ]:
!zip -r model.zip "/content/model"

In [ ]:
!zip -r results.zip "/content/results"

## Test Data Preprocessing

In [25]:
def preprocess_data(json_file_path):
    def clean_text(text):
      # Remove usernames
      text = re.sub(r'@[\w]+', '', text)
      # Convert to lowercase
      text = text.lower()
      # Replace emojis with their unicode description
      text = emoji.demojize(text, language='alias')
      # Hashtag segmentation
      text = re.sub(r'#(\w+)', lambda m: ' '.join(segment(m.group(1))), text)
      # Remove links
      text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
      # Remove unwanted '/'
      text = re.sub(r'/\s', '', text)
      # Remove punctuation except ' - / % & ... ¿ ?
      text = re.sub(r'[^\w\s\'%&¿?/-]', '', text)
      # Remove accented characters
      text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
      return text

    # function to remove data rows from dataframe if "yes" and "no" labels for each tweet are tied in label_task1 column

    # Load JSON data
    with open(json_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Extract data and clean tweets
    rows = []
    for key in data:
        item = data[key]
        row = [
            item['id_EXIST'],
            item['lang'],
            clean_text(item['tweet'])
        ]
        rows.append(row)
    # Write data to CSV
    with open('testdata.csv', 'w', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['id_EXIST', 'lang', 'tweet'])
        csvwriter.writerows(rows)

    df_test_data = pd.read_csv("testdata.csv")

    # Remove stopwords

    nltk.download('stopwords')

    # Remove stopwords from Spanish tweets
    spanish_stopwords = stopwords.words('spanish')
    df_test_data.loc[df_test_data['lang'] == 'es', 'tweet'] = df_test_data.loc[df_test_data['lang'] == 'es', 'tweet'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in spanish_stopwords]))

    # Remove stopwords from English tweets
    english_stopwords = stopwords.words('english')
    df_test_data.loc[df_test_data['lang'] == 'en', 'tweet'] = df_test_data.loc[df_test_data['lang'] == 'en', 'tweet'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in english_stopwords]))

    return df_test_data

In [26]:
def process_and_tokenize_test_data(df_test_data):
    # Set up tokenizer
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

    # Tokenize tweets and create attention masks
    encoded_tweets = tokenizer.batch_encode_plus(
        df_test_data['tweet'].tolist(),
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    return encoded_tweets


In [32]:
class CustomTestDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Placeholder tensor with the shape of the additional data [4]
        placeholder_additional_data = torch.zeros(4, dtype=torch.float)
        item["additional_data"] = placeholder_additional_data
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])


In [33]:
json_file_path = r"/N/u/aks14/Carbonate/Downloads/Applying_MLinCL_project/test/EXIST2023_test_clean.json"
df_test_cleaned = preprocess_data(json_file_path)
# df_test_cleaned.info()

[nltk_data] Downloading package stopwords to
[nltk_data]     /N/u/aks14/Carbonate/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
encoded_test_tweets = process_and_tokenize_test_data(df_test_cleaned)
df_test_cleaned.reset_index(drop=True, inplace=True)
test_dataset = CustomTestDataset(encoded_test_tweets)


## Evaluation on test data

In [51]:
test_predictions = trainer.predict(test_dataset)


/tmp/ipykernel_12357/3261244146.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


CLS output shape: torch.Size([16, 768])
Additional data shape: torch.Size([16, 4])
Combined output shape: torch.Size([16, 772])


CLS output shape: torch.Size([16, 768])
Additional data shape: torch.Size([16, 4])
Combined output shape: torch.Size([16, 772])
CLS output shape: torch.Size([16, 768])
Additional data shape: torch.Size([16, 4])
Combined output shape: torch.Size([16, 772])
CLS output shape: torch.Size([16, 768])
Additional data shape: torch.Size([16, 4])
Combined output shape: torch.Size([16, 772])
CLS output shape: torch.Size([16, 768])
Additional data shape: torch.Size([16, 4])
Combined output shape: torch.Size([16, 772])
CLS output shape: torch.Size([16, 768])
Additional data shape: torch.Size([16, 4])
Combined output shape: torch.Size([16, 772])
CLS output shape: torch.Size([16, 768])
Additional data shape: torch.Size([16, 4])
Combined output shape: torch.Size([16, 772])
CLS output shape: torch.Size([16, 768])
Additional data shape: torch.Size([16, 4])
Combined output shape: torch.Size([16, 772])
CLS output shape: torch.Size([16, 768])
Additional data shape: torch.Size([16, 4])
Combined output shape

CLS output shape: torch.Size([16, 768])
Additional data shape: torch.Size([16, 4])
Combined output shape: torch.Size([16, 772])
CLS output shape: torch.Size([16, 768])
Additional data shape: torch.Size([16, 4])
Combined output shape: torch.Size([16, 772])
CLS output shape: torch.Size([16, 768])
Additional data shape: torch.Size([16, 4])
Combined output shape: torch.Size([16, 772])
CLS output shape: torch.Size([16, 768])
Additional data shape: torch.Size([16, 4])
Combined output shape: torch.Size([16, 772])
CLS output shape: torch.Size([16, 768])
Additional data shape: torch.Size([16, 4])
Combined output shape: torch.Size([16, 772])
CLS output shape: torch.Size([16, 768])
Additional data shape: torch.Size([16, 4])
Combined output shape: torch.Size([16, 772])
CLS output shape: torch.Size([16, 768])
Additional data shape: torch.Size([16, 4])
Combined output shape: torch.Size([16, 772])
CLS output shape: torch.Size([16, 768])
Additional data shape: torch.Size([16, 4])
Combined output shape

In [41]:
predicted_labels = np.argmax(test_predictions.predictions, axis=1)


In [42]:
label_mapping_inverse = {0: 'NO', 1: 'YES'}
predicted_labels = [label_mapping_inverse[index] for index in predicted_labels]

In [43]:
print(predicted_labels)

['NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'YES', 'NO', 'YES', 'YES', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'YES', 'NO', 'NO', 'NO', 'NO', 'YES', 'YES', 'NO', 'YES', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'YES', 'NO', 'NO', 'YES', 'NO', 'YES', 'YES', 'YES', 'YES', 'YES', 'NO', 'YES', 'YES', 'NO', 'YES', 'YES', 'YES', 'YES', 'YES', 'YES', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'YES', 'YES', 'YES', 'NO', 'YES', 'YES', 'NO', 'NO', 'NO', 'YES', 'YES', 'YES', 'YES', 'YES', 'YES', 'NO', 'NO', 'NO', 'NO', 'YES', 'YES', 'NO', 'NO', 'NO', 'NO', 'YES', 'YES', 'NO', 'NO', 'NO', 'YES', 'NO', 'NO', 'YES', 'NO', 'YES', 'YES', 'NO', 'NO', 'YES', 'YES', 'YES', 'YES', 'NO', 'YES', 'NO', 'NO', 'NO', 'NO', 'YES', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'YES', 'NO', 'NO', 'YES', 'NO', 'YES', 'YES', 'YES', 'YES', 'YES', 'YES', 'YES', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'YES', 'NO', 'NO', 'YES',

In [52]:
output_df = pd.DataFrame({'id_EXIST': df_test_cleaned['id_EXIST'], 'predicted_label': predicted_labels})
output_df.to_csv('test_predictions.csv', index=False)